In [1]:
'''
Created on 30 July 2019 for Dish IOT demo

Regression models: How many more cycles an in-service device will last before it fails?

'''
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use("Agg")
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

# Setting seed for reproducibility - do this before importing keras.models.Sequential
np.random.seed(20190801)
PYTHONHASHSEED = 8012019
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM




# define path to save model
# model_path = '../Output/regression_model.h5'


In [2]:
# Mount the data location
dbutils.fs.unmount("/mnt/t3diiotdatastorage")

/mnt/t3diiotdatastorage has been unmounted.
 Out[ 11 ]: True

In [3]:
dbutils.fs.mount(
  source = "wasbs://production@t3diiotdatastorage.blob.core.windows.net",
  mount_point = "/mnt/t3diiotdatastorage",
  extra_configs = {"fs.azure.account.key.t3diiotdatastorage.blob.core.windows.net":dbutils.secrets.get(scope = "azure-key-vault-secrets", key = "processedtraindataaccountkey")})

Out[ 12 ]: True

In [4]:
dbutils.fs.ls("/mnt/t3diiotdatastorage")

Out[ 13 ]: 
[FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Device Data with Timestamps.csv', name='Device Data with Timestamps.csv', size=928420),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Device Data.csv', name='Device Data.csv', size=744580),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Sample Data.csv', name='Sample Data.csv', size=3982164),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/predict-transform-output.csv', name='predict-transform-output.csv', size=1080542),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/processed_train_fd001_with_rul.csv', name='processed_train_fd001_with_rul.csv', size=3548554),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/wrangled_data_folder/', name='wrangled_data_folder/', size=0)]

In [6]:
# spark_df = spark.read.csv("/mnt/t3diiotdatastorage/Device Data with Timestamps.csv", inferSchema=True, header=True)
spark_df = spark.read.csv("/mnt/t3diiotdatastorage/predict-transform-output.csv", inferSchema=True, header=True)

In [8]:
##################################
# Data Ingestion
##################################

# train_df = pd.read_csv('../Dataset/predict-transform-output.csv')
df = spark_df.toPandas()
df.columns = ['unit_number', 'device_timestamp', 'latency', 'voltage', 'availBw', 'vlanId', 'erroredSeconds',
                     'cpu', 'unique_sequence', 'rul']
df = train_df.sort_values(['unit_number','unique_sequence'])
df.head()

Out[ 15 ]: 
 unit_number device_timestamp latency ... cpu unique_sequence rul
0 1 1564349324 261 ... 29 1 258
1 1 1564349325 259 ... 73 2 257
2 1 1564349326 258 ... 58 3 256
3 1 1564349327 259 ... 36 4 255
4 1 1564349328 256 ... 44 5 254

[5 rows x 10 columns]

In [9]:
# 
from sklearn.model_selection import KFold
k = 5
kf = KFold(n_splits=k, random_state=201809, shuffle=True)
unit_number = df.unit_number.unique()
kf.get_n_splits(unit_number)

fold_indices = []
for train_indices, test_indices in kf.split(unit_number):
    fold_indices.append((train_indices, test_indices))

# just build for one fold first; then we can put in a loop
train_ids = unit_number[fold_indices[0][0]]
test_ids = unit_number[fold_indices[0][1]]

train_df = df[df['unit_number'].isin(train_ids)]
test_df = df[df['unit_number'].isin(test_ids)]

train_df = train_df.sort_values(['unit_number', 'unique_sequence'])
test_df = test_df.sort_values(['unit_number', 'unique_sequence'])

In [10]:
# Lets check number of unique devices - Train set
print(train_df.shape)
print(len(train_df['unit_number'].unique()))
print(train_df['unit_number'].unique())
print(train_df.head())

# Lets check number of unique devices - Test set
print(test_df.shape)
print(len(test_df['unit_number'].unique()))
print(test_df['unit_number'].unique())
print(test_df.head())

(20149, 10)
80
[ 1 2 3 5 6 8 9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25 27 28
 29 30 31 33 34 35 36 37 38 40 41 43 44 46 47 48 50 51 52 53 54 55 56 57
 58 59 60 61 62 63 64 65 66 67 69 70 71 73 74 75 77 78 79 82 84 85 86 90
 91 93 94 95 96 97 98 99]
 unit_number device_timestamp latency ... cpu unique_sequence rul
0 1 1564349324 261 ... 29 1 258
1 1 1564349325 259 ... 73 2 257
2 1 1564349326 258 ... 58 3 256
3 1 1564349327 259 ... 36 4 255
4 1 1564349328 256 ... 44 5 254

[5 rows x 10 columns]
(4571, 10)
20
[ 4 7 19 26 32 39 42 45 49 68 72 76 80 81 83 87 88 89
 92 100]
 unit_number device_timestamp latency ... cpu unique_sequence rul
734 4 1564349324 262 ... 65 1 271
735 4 1564349325 264 ... 31 2 270
736 4 1564349326 269 ... 65 3 269
737 4 1564349327 275 ... 54 4 268
738 4 1564349328 262 ... 51 5 267

[5 rows x 10 columns]

In [11]:
##################################
# Data Preprocessing
##################################

#######
# TRAIN
#######

# generate label columns for training data just in case we need to create binary classification model
# answer the question: is a specific engine going to fail within w1 cycles?
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['rul'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['rul'] <= w0, 'label2'] = 2
train_df.head()

Out[ 19 ]: 
 unit_number device_timestamp latency ... rul label1 label2
0 1 1564349324 261 ... 258 0 0
1 1 1564349325 259 ... 257 0 0
2 1 1564349326 258 ... 256 0 0
3 1 1564349327 259 ... 255 0 0
4 1 1564349328 256 ... 254 0 0

[5 rows x 12 columns]

In [12]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['rul'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['rul'] <= w0, 'label2'] = 2

In [13]:
# MinMax normalization (from 0 to 1)
train_df['unique_sequence_norm'] = train_df['unique_sequence']
cols_normalize = train_df.columns.difference(['unit_number', 'device_timestamp', 'unique_sequence', 'rul','label1','label2'])

min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

/databricks/python/lib/python3.5/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by MinMaxScaler.
 return self.partial_fit(X, y)
 Out[ 21 ]: 
 unit_number ... unique_sequence_norm
0 1 ... 0.000000
1 1 ... 0.001908
2 1 ... 0.003817
3 1 ... 0.005725
4 1 ... 0.007634

[5 rows x 13 columns]

In [14]:
# MinMax normalization (from 0 to 1) for test data
test_df['unique_sequence_norm'] = test_df['unique_sequence']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

Out[ 22 ]: 
 unit_number ... unique_sequence_norm
0 4 ... 0.000000
1 4 ... 0.001908
2 4 ... 0.003817
3 4 ... 0.005725
4 4 ... 0.007634

[5 rows x 13 columns]

In [15]:
# pick a large window size of 100 cycles
sequence_length = 100

# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

In [16]:
# pick the feature columns 
sequence_cols = ['latency', 'voltage', 'availBw', 'vlanId', 'erroredSeconds', 'cpu', 'unique_sequence_norm']
sequence_cols

Out[ 24 ]: 
['latency',
 'voltage',
 'availBw',
 'vlanId',
 'erroredSeconds',
 'cpu',
 'unique_sequence_norm']

In [17]:
# TODO for debug 
# val is a list of 192 - 50 = 142 bi-dimensional array (50 rows x 25 columns)
# val=list(gen_sequence(train_df[train_df['unit_number']==1], sequence_length, sequence_cols))
# print(len(val))

# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['unit_number']==id], sequence_length, sequence_cols)) 
           for id in train_df['unit_number'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

(12149, 100, 7)

In [18]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]] 
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['unit_number']==id], sequence_length, ['rul']) 
             for id in train_df['unit_number'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

Out[ 26 ]: (12149, 1)

In [19]:
# generator for the sequences in test set
# transform each id of the test dataset in a sequence
seq_gen_test = (list(gen_sequence(test_df[test_df['unit_number']==id], sequence_length, sequence_cols)) 
           for id in test_df['unit_number'].unique())

# generate sequences and convert to numpy array
seq_array_test = np.concatenate(list(seq_gen_test)).astype(np.float32)
print(seq_array_test.shape)
# generate labels
label_gen_test = [gen_labels(test_df[test_df['unit_number']==id], sequence_length, ['rul']) 
             for id in test_df['unit_number'].unique()]

label_array_test = np.concatenate(label_gen_test).astype(np.float32)
print(label_array_test.shape)

(2571, 100, 7)
(2571, 1)

In [20]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [21]:
##################################
# Modeling
##################################

# Next, we build a deep network. 
# The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. 
# Dropout is also applied after each LSTM layer to control overfitting. 
# Final layer is a Dense output layer with single unit and linear activation since this is a regression problem.
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])

print(model.summary())

# fit the network
# history = model.fit(seq_array, label_array, epochs=100, batch_size=100, validation_split=0.1, verbose=2,
#           callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
#                        keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
#           )

# history = model.fit(seq_array, label_array, epochs=100, batch_size=100, validation_split=0.1, verbose=2,
#           callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')]
#           )

history = model.fit(seq_array, label_array, epochs=100, batch_size=100, validation_data=(seq_array_test, label_array_test), verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')]
          )

# list all data in history
print(history.history.keys())

_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
lstm_3 (LSTM) (None, 100, 100) 43200 
_________________________________________________________________
dropout_3 (Dropout) (None, 100, 100) 0 
_________________________________________________________________
lstm_4 (LSTM) (None, 50) 30200 
_________________________________________________________________
dropout_4 (Dropout) (None, 50) 0 
_________________________________________________________________
dense_2 (Dense) (None, 1) 51 
_________________________________________________________________
activation_2 (Activation) (None, 1) 0 
=================================================================
Total params: 73,451
Trainable params: 73,451
Non-trainable params: 0
_________________________________________________________________
None
Train on 12149 samples, validate on 2571 samples
Epoch 1/100
 - 25s - loss: 15579.5036 - mean_absolute_error: 91.6668 - r2_keras: -1.1583e+00 - val_loss: 9350.2056 - val_mean_absolute_error: 70.9144 - val_r2_keras: -6.5146e+00
Epoch 2/100
 - 22s - loss: 14443.4513 - mean_absolute_error: 86.3920 - r2_keras: -9.9589e-01 - val_loss: 8552.8668 - val_mean_absolute_error: 66.5440 - val_r2_keras: -5.7511e+00
Epoch 3/100
 - 22s - loss: 13474.2655 - mean_absolute_error: 82.0382 - r2_keras: -8.6067e-01 - val_loss: 7826.7312 - val_mean_absolute_error: 62.7307 - val_r2_keras: -5.0876e+00
Epoch 4/100
 - 23s - loss: 12574.1632 - mean_absolute_error: 78.1407 - r2_keras: -7.3664e-01 - val_loss: 7178.1870 - val_mean_absolute_error: 59.5061 - val_r2_keras: -4.5296e+00
Epoch 5/100
 - 22s - loss: 11746.1574 - mean_absolute_error: 74.7312 - r2_keras: -6.1726e-01 - val_loss: 6590.3609 - val_mean_absolute_error: 56.7908 - val_r2_keras: -4.0633e+00
Epoch 6/100
 - 22s - loss: 10964.1925 - mean_absolute_error: 71.7243 - r2_keras: -5.0801e-01 - val_loss: 6075.4618 - val_mean_absolute_error: 54.6486 - val_r2_keras: -3.7000e+00
Epoch 7/100
 - 22s - loss: 10265.0825 - mean_absolute_error: 69.2882 - r2_keras: -4.1189e-01 - val_loss: 5641.1971 - val_mean_absolute_error: 53.1008 - val_r2_keras: -3.4441e+00
Epoch 8/100
 - 22s - loss: 9646.1467 - mean_absolute_error: 67.2253 - r2_keras: -3.2692e-01 - val_loss: 5278.8438 - val_mean_absolute_error: 52.0753 - val_r2_keras: -3.2895e+00
Epoch 9/100
 - 22s - loss: 9115.9408 - mean_absolute_error: 65.7530 - r2_keras: -2.5116e-01 - val_loss: 4991.1447 - val_mean_absolute_error: 51.5280 - val_r2_keras: -3.2370e+00
Epoch 10/100
 - 22s - loss: 8668.8878 - mean_absolute_error: 64.7746 - r2_keras: -1.8841e-01 - val_loss: 4778.9459 - val_mean_absolute_error: 51.4282 - val_r2_keras: -3.2836e+00
Epoch 11/100
 - 22s - loss: 8272.1621 - mean_absolute_error: 64.1355 - r2_keras: -1.3592e-01 - val_loss: 4637.1676 - val_mean_absolute_error: 51.7547 - val_r2_keras: -3.4260e+00
Epoch 12/100
 - 23s - loss: 7967.9629 - mean_absolute_error: 64.0105 - r2_keras: -9.4377e-02 - val_loss: 4565.3462 - val_mean_absolute_error: 52.4550 - val_r2_keras: -3.6546e+00
Epoch 13/100
 - 22s - loss: 7759.3840 - mean_absolute_error: 63.9505 - r2_keras: -6.5274e-02 - val_loss: 4554.5798 - val_mean_absolute_error: 52.8379 - val_r2_keras: -3.7709e+00
Epoch 14/100
 - 22s - loss: 7459.0132 - mean_absolute_error: 62.8381 - r2_keras: -2.4094e-02 - val_loss: 4509.1839 - val_mean_absolute_error: 54.1525 - val_r2_keras: -4.1941e+00
Epoch 15/100
 - 23s - loss: 6248.1602 - mean_absolute_error: 51.2314 - r2_keras: 0.1460 - val_loss: 3001.1004 - val_mean_absolute_error: 33.4398 - val_r2_keras: -1.2288e+00
Epoch 16/100
 - 22s - loss: 5668.7424 - mean_absolute_error: 46.8387 - r2_keras: 0.2257 - val_loss: 2541.6623 - val_mean_absolute_error: 30.5420 - val_r2_keras: -8.4102e-01
Epoch 17/100
 - 23s - loss: 5176.1323 - mean_absolute_error: 43.0511 - r2_keras: 0.2921 - val_loss: 2410.7518 - val_mean_absolute_error: 32.2252 - val_r2_keras: -9.4996e-01
Epoch 18/100
 - 22s - los

In [22]:
# define path to save model
model_path = "/mnt/t3diiotdatastorage/rul_model.h5"
model_path =  
dbutils.fs.ls("/FileStore/tables/rul_model.h5")
model.save(model_path)

--------------------------------------------------------------------------- 
 OSError Traceback (most recent call last)
 <command-4255519565090214> in <module> () 
 1 # define path to save model 
 2 model_path = "/mnt/t3diiotdatastorage/rul_model.h5" 
 ----> 3 model . save ( model_path ) 

 /databricks/python/lib/python3.5/site-packages/keras/engine/network.py in save (self, filepath, overwrite, include_optimizer) 
 1088 raise NotImplementedError 
 1089 from . . models import save_model 
 -> 1090 save_model ( self , filepath , overwrite , include_optimizer ) 
 1091 
 1092 def save_weights ( self , filepath , overwrite = True ) : 

 /databricks/python/lib/python3.5/site-packages/keras/engine/saving.py in save_model (model, filepath, overwrite, include_optimizer) 
 377 opened_new_file = False 
 378 
 --> 379 f = h5dict ( filepath , mode = 'w' ) 
 380 
 381 try : 

 /databricks/python/lib/python3.5/site-packages/keras/utils/io_utils.py in __init__ (self, path, mode) 
 184 self . _is_file = False 
 185 elif isinstance ( path , str ) : 
 --> 186 self . data = h5py . File ( path , mode = mode ) 
 187 self . _is_file = True 
 188 elif isinstance ( path , dict ) : 

 /databricks/python/lib/python3.5/site-packages/h5py/_hl/files.py in __init__ (self, name, mode, driver, libver, userblock_size, swmr, rdcc_nslots, rdcc_nbytes, rdcc_w0, track_order, **kwds) 
 392 fid = make_fid(name, mode, userblock_size,
 393 fapl , fcpl = make_fcpl ( track_order = track_order ) , 
 --> 394 swmr=swmr)
 395 
 396 if swmr_support : 

 /databricks/python/lib/python3.5/site-packages/h5py/_hl/files.py in make_fid (name, mode, userblock_size, fapl, fcpl, swmr) 
 174 fid = h5f . create ( name , h5f . ACC_EXCL , fapl = fapl , fcpl = fcpl ) 
 175 elif mode == 'w' : 
 --> 176 fid = h5f . create ( name , h5f . ACC_TRUNC , fapl = fapl , fcpl = fcpl ) 
 177 elif mode == 'a' : 
 178 # Open in append mode (read/write). 

 /databricks/python/lib/python3.5/site-packages/h5py/_objects.cpython-35m-x86_64-linux-gnu.so in h5py._objects.with_phil.wrapper () 

 /databricks/python/lib/python3.5/site-packages/h5py/_objects.cpython-35m-x86_64-linux-gnu.so in h5py._objects.with_phil.wrapper () 

 /databricks/python/lib/python3.5/site-packages/h5py/h5f.cpython-35m-x86_64-linux-gnu.so in h5py.h5f.create () 

 OSError : Unable to create file (unable to open file: name = '/mnt/t3diiotdatastorage/rul_model.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [23]:
model_path = "rul_model.h5"
model.save(model_path)

In [24]:
dbutils.fs.ls("/mnt/t3diiotdatastorage")

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-4255519565090219> in <module> () 
 ----> 1 dbutils . fs . pwd ( ) 

 AttributeError : 'FSHandler' object has no attribute 'pwd'

In [25]:
# training metrics
train_scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)

200/12149 [..............................] - ETA: 4s
 400/12149 [..............................] - ETA: 3s
 600/12149 [>.............................] - ETA: 3s
 800/12149 [>.............................] - ETA: 3s
 1000/12149 [=>............................] - ETA: 3s
 1200/12149 [=>............................] - ETA: 3s
 1400/12149 [==>...........................] - ETA: 3s
 1600/12149 [==>...........................] - ETA: 3s
 1800/12149 [===>..........................] - ETA: 3s
 2000/12149 [===>..........................] - ETA: 3s
 2200/12149 [====>.........................] - ETA: 3s
 2400/12149 [====>.........................] - ETA: 3s
 2600/12149 [=====>........................] - ETA: 3s
 2800/12149 [=====>........................] - ETA: 3s
 3000/12149 [======>.......................] - ETA: 3s
 3200/12149 [======>.......................] - ETA: 3s
 3400/12149 [=======>......................] - ETA: 2s
 3600/12149 [=======>......................] - ETA: 2s
 3800/12149 [========>.....................] - ETA: 2s
 4000/12149 [========>.....................] - ETA: 2s
 4200/12149 [=========>....................] - ETA: 2s
 4400/12149 [=========>....................] - ETA: 2s
 4600/12149 [==========>...................] - ETA: 2s
 4800/12149 [==========>...................] - ETA: 2s
 5000/12149 [===========>..................] - ETA: 2s
 5200/12149 [===========>..................] - ETA: 2s
 5400/12149 [============>.................] - ETA: 2s
 5600/12149 [============>.................] - ETA: 2s
 5800/12149 [=============>................] - ETA: 2s
 6000/12149 [=============>................] - ETA: 2s
 6200/12149 [==============>...............] - ETA: 2s
 6400/12149 [==============>...............] - ETA: 2s
 6600/12149 [===============>..............] - ETA: 1s
 6800/12149 [===============>..............] - ETA: 1s
 7000/12149 [================>.............] - ETA: 1s
 7200/12149 [================>.............] - ETA: 1s
 7400/12149 [=================>............] - ETA: 1s
 7600/12149 [=================>............] - ETA: 1s
 7800/12149 [==================>...........] - ETA: 1s
 8000/12149 [==================>...........] - ETA: 1s
 8200/12149 [===================>..........] - ETA: 1s
 8400/12149 [===================>..........] - ETA: 1s
 8600/12149 [====================>.........] - ETA: 1s
 8800/12149 [====================>.........] - ETA: 1s
 9000/12149 [=====================>........] - ETA: 1s
 9200/12149 [=====================>........] - ETA: 1s
 9400/12149 [======================>.......] - ETA: 0s
 9600/12149 [======================>.......] - ETA: 0s
 9800/12149 [=======================>......] - ETA: 0s
10000/12149 [=======================>......] - ETA: 0s
10200/12149 [========================>.....] - ETA: 0s
10400/12149 [========================>.....] - ETA: 0s
10600/12149 [=========================>....] - ETA: 0s
10800/12149 [=========================>....] - ETA: 0s
11000/12149 [==========================>...] - ETA: 0s
11200/12149 [==========================>...] - ETA: 0s
11400/12149 [===========================>..] - ETA: 0s
11600/12149 [===========================>..] - ETA: 0s
11800/12149 [============================>.] - ETA: 0s
12000/12149 [============================>.] - ETA: 0s
12149/12149 [==============================] - 4s 348us/step

MAE: 11.344360111750579

R^2: 0.8838620221372474

In [26]:
# training metrics
print('\nTrain MAE: {}'.format(train_scores[1]))
print('\nTrain R^2: {}'.format(train_scores[2]))

Train MAE: 11.344360111750579

Train R^2: 0.8838620221372474

In [27]:
# testing metrics
test_scores = model.evaluate(seq_array_test, label_array_test, verbose=1, batch_size=200)

200/2571 [=>............................] - ETA: 0s
 400/2571 [===>..........................] - ETA: 0s
 600/2571 [======>.......................] - ETA: 0s
 800/2571 [========>.....................] - ETA: 0s
1000/2571 [==========>...................] - ETA: 0s
1200/2571 [=============>................] - ETA: 0s
1400/2571 [===============>..............] - ETA: 0s
1600/2571 [=================>............] - ETA: 0s
1800/2571 [====================>.........] - ETA: 0s
2000/2571 [======================>.......] - ETA: 0s
2200/2571 [========================>.....] - ETA: 0s
2400/2571 [===========================>..] - ETA: 0s
2571/2571 [==============================] - 1s 353us/step

In [28]:
# testing metrics
print('\nTest MAE: {}'.format(test_scores[1]))
print('\nTest R^2: {}'.format(test_scores[2]))

Test MAE: 11.433075410946016

Test R^2: 0.8595819477439622

In [29]:
dbutils.fs.ls("/mnt/t3diiotdatastorage")

Out[ 39 ]: 
[FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Device Data with Timestamps.csv', name='Device Data with Timestamps.csv', size=928420),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Device Data.csv', name='Device Data.csv', size=744580),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/Sample Data.csv', name='Sample Data.csv', size=3982164),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/predict-transform-output.csv', name='predict-transform-output.csv', size=1080542),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/processed_train_fd001_with_rul.csv', name='processed_train_fd001_with_rul.csv', size=3548554),
 FileInfo(path='dbfs:/mnt/t3diiotdatastorage/wrangled_data_folder/', name='wrangled_data_folder/', size=0)]

In [30]:
dbutils.fs.ls("/FileStore/tables/")

Out[ 47 ]: [FileInfo(path='dbfs:/FileStore/tables/rul_model.h5', name='rul_model.h5', size=612664)]